# Diagonally Dominant Tridiagonal Matrices

To find the $D^2$-spline g we have to solve the triangular system:
$$\left[\begin{array}{ccccc}
4 & 1 & & & \\
1 & 4 & 1 & & \\
& \ddots & \ddots & \ddots & \\
& & 1 & 4 & 1 \\
& & & 1 & 4
\end{array}\right]\left[\begin{array}{c}
\mu_{2} \\
\mu_{3} \\
\vdots \\
\mu_{n-1} \\
\mu_{n}
\end{array}\right]=\frac{6}{h^{2}}\left[\begin{array}{c}
\delta^{2} y_{2}-\mu_{1} \\
\delta^{2} y_{3} \\
\vdots \\
\delta^{2} y_{n-1} \\
\delta^{2} y_{n}-\mu_{n+1}
\end{array}\right], \delta^{2} y_{i}:=y_{i+1}-2 y_{i}+y_{i-1} $$

Consider solving a general tridiagonal linear system $Ax = b$ where A = tridiag($a_i, d_i, c_i$) ∈ $\mathbb{C}^{n×n}$. Instead of using Gaussian elimination directly, we can construct two matrices L and U such that $A = LU$. Since $Ax = LUx = b$ we can find x by solving two systems $Lz = b$ and $U x = z$. Moreover L and U are both triangular and bidiagonal, and if in addition they are nonsingular the two systems can be solved easily without using elimination. In our case we write the product $A = LU$ in the form:

$$\left[\begin{array}{cccc}
d_{1} & c_{1} & & \\
a_{1} & d_{2} & c_{2} & & \\
& \ddots & \ddots & \ddots & \\
& & a_{n-2} & d_{n-1} & c_{n-1} \\
& & & a_{n-1} & d_{n}
\end{array}\right]=\left[\begin{array}{ccc}
1 & & & \\
l_{1} & 1 & & \\
& \ddots & \ddots & \\
& & l_{n-1} & 1
\end{array}\right]\left[\begin{array}{cccc}
u_{1} & c_{1} & & \\
& \ddots & \ddots & \\
& & u_{n-1} & c_{n-1} \\
& & & u_{n}
\end{array}\right]$$

To find L and U we first consider the case n = 3 then we have from the above system:

$$\left[\begin{array}{lll}
d_{1} & c_{1} & 0 \\
a_{1} & d_{2} & c_{2} \\
0 & a_{2} & d_{3}
\end{array}\right]=\left[\begin{array}{ccc}
1 & 0 & 0 \\
l_{1} & 1 & 0 \\
0 & l_{2} & 1
\end{array}\right]\left[\begin{array}{ccc}
u_{1} & c_{1} & 0 \\
0 & u_{2} & c_{2} \\
0 & 0 & u_{3}
\end{array}\right]=\left[\begin{array}{ccc}
u_{1} & c_{1} & 0 \\
l_{1} u_{1} & l_{1} c_{1}+u_{2} & c_{2} \\
0 & l_{2} u_{2} & l_{2} c_{2}+u_{3}
\end{array}\right] $$

and the systems $Lz = b$ and $Ux = z$ can be written:

$$\left[\begin{array}{lll}
1 & 0 & 0 \\
l_{1} & 1 & 0 \\
0 & l_{2} & 1
\end{array}\right]\left[\begin{array}{l}
z_{1} \\
z_{2} \\
z_{3}
\end{array}\right]=\left[\begin{array}{l}
b_{1} \\
b_{2} \\
b_{3}
\end{array}\right], \quad\left[\begin{array}{lll}
u_{1} & c_{1} & 0 \\
0 & u_{2} & c_{2} \\
0 & 0 & u_{3}
\end{array}\right]\left[\begin{array}{l}
x_{1} \\
x_{2} \\
x_{3}
\end{array}\right]=\left[\begin{array}{l}
z_{1} \\
z_{2} \\
z_{3}
\end{array}\right] $$

Comparing elements we find: 

$$\begin{array}{l}
u_{1}=d_{1}, \quad l_{1}=a_{1} / u_{1}, \quad u_{2}=d_{2}-l_{1} c_{1}, \quad l_{2}=a_{2} / u_{2}, \quad u_{3}=d_{3}-l_{2} c_{2} \\
z_{1}=b_{1}, \quad z_{2}=b_{2}-l_{1} z_{1}, \quad z_{3}=b_{3}-l_{2} z_{2} \\
x_{3}=z_{3} / u_{3}, \quad x_{2}=\left(z_{2}-c_{2} x_{3}\right) / u_{2}, \quad x_{1}=\left(z_{1}-c_{1} x_{2}\right) / u_{1}
\end{array} $$

In general, if:

$$ u_{1}=d_{1}, \quad l_{k}=a_{k} / u_{k}, \quad u_{k+1}=d_{k+1}-l_{k} c_{k}, \quad k=1,2, \ldots, n-1$$

then $A = LU$. If $u_1, u_2,...,u_{n−1}$ are nonzero then the above system is well defined. If in addition $u_n \neq 0$ then we can solve $Lz = b$ and $Ux = z$ for z and x. We formulate this as two algorithms:

In [1]:
from numpy import *
import matplotlib.pyplot as plt



def trifactor(a, d, c):
    """
    Compute the entries in the LU-factorization of a tridiagonal matrix A using algorithm 1.8. 
    Returns the sequence l on the sub-diagonal of L, and the sequence u on the diagonal of U. 
    
    a: The n-1 entries on the sub-diagonal of A
    d: The n entries on the diagonal of A
    c: The n-1 entries on the super-diagonal of A
    """
    u, l = zeros_like(d), zeros_like(a)
    u[0] = d[0]
    for k in range(len(a)):
        l[k] = a[k]/u[k]
        u[k+1] = d[k+1] - l[k]*c[k]
    return l, u

In [2]:
def trisolve(l, u, c, b):
    """
    Compute the solution to a system on the form A*x=b with A a tridiagonal matrix, based on the LU-factorization of A 
    as given by the function trifactor (using algorithm 1.9). 
    The input vector b will be updated with the solution x (i.e. in-place).
    
    l: The entries on the sub-diagonal of L in the LU-factorization of A
    u: The entries on the diagonal of U in the LU-factorization of A
    c: The entries on the super-diagonal of U in the LU-factorization of A (this equals the entries on the super-diagonal of A)
    b: The right hand side
    """
    n = shape(b)[0]
    for k in range(1, n):
        b[k] -= l[k-1]*b[k - 1]
    b[n-1] /= u[n-1]
    for k in range(n-2,-1,-1):
        b[k] -= c[k]*b[k + 1]
        b[k] /= u[k]

Since division by zero can occur, the algorithms will not work in general, but for tridiagonal strictly diagonally dominant systems we have s a unique LU factorization. The number of arithmetic operations to compute the LU factorization of a tridiagonal matrix of order n using is 3n−3, while the number of arithmetic operations for Algorithm trisolve is r(5n−4), where r is the number of right-hand sides. This means that the complexity to solve a tridiagonal system is O(n), or
more precisely 8n − 7 when r = 1, and this number only grows linearly with n.